In [ ]:
# 120. Triangle
class Solution:
    def minimumTotal(self, triangle: List[List[int]]) -> int:
#         记忆化搜索的好处是通常能将指数级别的复杂度降低到多项式级别
        # print(triangle[1][0])
        if not triangle or not triangle[0]:
            return 0
        return self.DFS(triangle,0,0,{})
    def DFS(self,triangle,x,y,memo):
        # global mini
        if x == len(triangle):
            return 0
#         如果找过了，就不用再找了，直接把之前找到的值返回
        if (x, y) in memo:
            return memo[(x, y)]
        
        left = self.DFS(triangle,x+1,y,memo)
        right = self.DFS(triangle,x+1,y+1,memo)
        res = min(left,right) + triangle[x][y]
        memo[(x,y)] = res
        return res
# 注意这个memo是从最底下一路记上来的，先访问了一整条路径到地下，然后从这路一路回溯上来记忆
# 为什么从指数级别变成了多项式呢，有记忆化之后少走了很多重复路径，
# 每一个点往下递归只走一次，每个点往左右看一次，n个点是2n，假设深度为n
# 那么复杂度就是O(2n^2) -> O(n^2)

# 底下是这道题的多种参考解法，前面两种会超时
五种方法： 
    方法1: Traverse （无返回值） 
    方法2: Divide and Conquer （带返回值） 
    方法3: DFS+memo （加cache） 
    方法4: DP，（填表格）
    方法5: DP + rolling array 优化空间 （求余大法）


class Solution:
    """
    @param triangle: a list of lists of integers
    @return: An integer, minimum path sum
    """
    # 1: Traverse O(2^n)
    def minimumTotalTraverse(self, triangle):
        # 老大拿记事本（变量）下基层，没有返回值
        self.minRes = float('inf')
        self.traversal(triangle, 0, 0, 0)
        return self.minRes
    def traversal(self, triangle, x, y, total):
        if x == len(triangle):
            self.minRes = min(self.minRes, total)
            return
        self.traversal(triangle, x+1, y, total+triangle[x][y])
        self.traversal(triangle, x+1, y+1, total+triangle[x][y])
    
    # 2: Divide and Conquer O(2^n)
    def minimumTotalDQ(self, triangle):   
        # 派小弟出去干活，带回来返回值
        return self.divdeConquer(triangle, 0, 0)
    def divdeConquer(self, triangle, x, y):
        if x == len(triangle):
            return 0
        left = self.divdeConquer(triangle, x+1, y)
        right = self.divdeConquer(triangle, x+1, y+1)
        return min(left, right) + triangle[x][y]
    
    #3: DFS + memo O(n^2)， 加cache统一套路
    def minimumTotalMemo(self, triangle):
        cache = {}
        return self.DFSmemo(triangle, 0, 0, cache)
    def DFSmemo(self, triangle, x, y, cache):
        if (x, y) in cache: return cache[(x,y)]
        if x == len(triangle): 
            return 0
        left = self.DFSmemo(triangle, x+1, y, cache)
        right = self.DFSmemo(triangle, x+1, y+1, cache)
        res =  min(left, right) + triangle[x][y]
        cache[(x,y)] = res
        return res
    
    # DP, Time complexity O(n^2), space complexity: O(n^2) 
    def minimumTotalDP(self, triangle):
        # dp[i][j]: minpathSum from top to triangle[i][j] 
        # dp[i][j] = min(dp[i-1][j-1], dp[i-1][j])+triangle[i][j]
        # initial dp[0][0] = triangle[0][0]，利用递推公式搞不出来就需要初始化
        # result: min(dp[n-1])
        n = len(triangle)
        dp = [[float('inf')] * (i+1) for i in range(n)]
        dp[0][0] = triangle[0][0]
        for i in range(1, n):
            for j in range(i+1):
                if j == 0:
                    dp[i][j] = dp[i-1][j] + triangle[i][j]
                elif j == i:
                    dp[i][j] = dp[i-1][j-1] + triangle[i][j]
                else:
                    dp[i][j] = min(dp[i-1][j-1], dp[i-1][j])+triangle[i][j]
        return min(dp[n-1])
    
    # DP, O(n^2) + rolling array O(n) space 优化空间, 求余大法好
    def minimumTotalDPOpt(self, triangle):
        n = len(triangle)
        dp = [[float('inf')] * n for _ in range(2)]
        dp[0][0] = triangle[0][0]
        i = 0
        for i in range(1, n):
            for j in range(i+1):
                if j == 0:
                    dp[i%2][j] = dp[(i-1)%2][j] + triangle[i][j]
                elif j == i:
                    dp[i%2][j] = dp[(i-1)%2][j-1] + triangle[i][j]
                else:
                    dp[i%2][j] = min(dp[(i-1)%2][j-1], dp[(i-1)%2][j])+triangle[i][j]
        return min(dp[i%2])

In [ ]:
# 64. Minimum Path Sum
# triangle的完全相同类型变种题
class Solution:
    def minPathSum(self, grid: List[List[int]]) -> int:
        if not grid or not grid[0]:
            return 0
        memo = {}
        self.dfs(grid,0,0,memo)
        return memo[(0,0)]
    def dfs(self,grid,x,y,memo):
        if (x, y) in memo:
            return memo[(x, y)]
        if x == len(grid) - 1 and y == len(grid[0]) - 1:
            memo[(x,y)] = grid[x][y]
            return grid[x][y]
#         如果找过了，就不用再找了，直接把之前找到的值返回
        
# 注意这里因为我同时存在两个需要进行判断的边界条件,(和triangle一个进行判断的边界条件不同),因此我需要
# 分别对x和y进行判断，这里引入两个if
# 这是一种自底而上的写法，先通过递归到达最后一层再层层出来
        if x < len(grid) - 1: 
            left = self.dfs(grid,x+1,y,memo)
        else:
            left = sys.maxsize
        if y < len(grid[0]) - 1:
            right = self.dfs(grid,x,y+1,memo)
        else:
            right = sys.maxsize
        
        res = min(left,right) + grid[x][y]
        memo[(x,y)] = res
        
        return res

In [ ]:
# 62. Unique Paths
# 只能向右走或者向下走两种情况，第一反应就是recursion
# 这里Method 1 采用的是DFS
class Solution:
    def uniquePaths(self, m: int, n: int) -> int:
        visited_path = {}
        return self.recur(0,0, visited_path,m ,n)
    
    def recur(self, i, j, visited_path,m ,n):
        if i == m-1 or j == n-1:
            return 1

        if (i, j) in visited_path:
            return visited_path[(i, j)]
        
        right = self.recur(i, j+1, visited_path,m ,n)
        down = self.recur(i+1, j, visited_path,m ,n)
        visited_path[(i,j)] =  right + down  
        return visited_path[(i,j)] #为啥这里返回的是visited_path[(i,j)]
# 同理还可以Method 2 采用DP来写
# 画一个自己的例子，我自己是4x4的一个例子，第一行和第一列都是全1，然后中间都是左边和上面的和
class Solution:
    def uniquePaths(self, m: int, n: int) -> int:
        dp = [[0 for i in range(n)] for j in range(m)]
        for i in range(m):
            for j in range(n):
                if i == 0 or j == 0:
                    dp[i][j] = 1
                else:
                    dp[i][j] = dp[i - 1][j] + dp[i][j - 1]
        return dp[m - 1][n - 1]

In [ ]:
# 63. Unique Paths II
# 这道题不同的地方在于对于第一列和第一行不再是全赋值为1了
# 而是取决于其上一个有无障碍物的情况，对于中心位置仍然是一样的
# 对于有障碍物直接让dp的值为0
# 其复杂度为O(mn)
class Solution:
    def uniquePathsWithObstacles(self, obstacleGrid: List[List[int]]) -> int:
        dp = [[0 for i in range(len(obstacleGrid[0]))] for j in range(len(obstacleGrid))]
        # if obstacleGrid[0][0] == 1:
        #     return 0
        # dp[0][0] = 1
        for i in range(len(obstacleGrid)):
            for j in range(len(obstacleGrid[0])):
                if obstacleGrid[i][j] == 1:
                    dp[i][j] = 0
                else:
                    # if i > 0:
                    #     dp[i][j] += dp[i - 1][j]
                    # if j > 0:
                    #     dp[i][j] += dp[i][j - 1]
                    if i == 0 and j == 0:
                        dp[i][j] = 1
                    elif i == 0:
                        dp[i][j] = dp[i][j - 1]
                    elif j == 0:
                        dp[i][j] = dp[i - 1][j]
                    else:
                        dp[i][j] = dp[i - 1][j] + dp[i][j - 1]
        return dp[len(obstacleGrid) - 1][len(obstacleGrid[0]) - 1]

In [ ]:
# 198. House Robber
# 注意不一定是要相隔一个房子，可以相隔两个房子之类的
# 因此这里是不能采用奇偶方法来写的
class Solution:
    def rob(self, nums: List[int]) -> int:
        rob = [None] *(len(nums))
        n = len(nums)
        if n is 0:
            return 0 
        for i in range(0,len(nums)):
            if i is 0:
                rob[i] = nums[0]
            elif i is 1:
                rob[i] = max(nums[0],nums[1])
            elif i >= 2:
                rob[i] = max(nums[i]+rob[i-2],rob[i-1])
        return rob[-1]

In [ ]:
# 213. House Robber II
# 算两次，第一次去掉第一座房子，第二次去掉最后一座房子，这样能避免第一座最后一座都被抢的情况。应该还比较简单易懂。
# 然后在两次中取最大值返回
class Solution:
    def rob(self, nums: List[int]) -> int:
        if len(nums) == 1:
            return nums[0]
        elif len(nums) == 2:
            return max(nums[0],nums[1])
        else:
            return max(self.dp(nums[1:]),self.dp(nums[:len(nums) - 1]))
    def dp(self,nums):
        rob = [None] *(len(nums))
        n = len(nums)
        if n is 0:
            return 0 
        for i in range(0,len(nums)):
            if i is 0:
                rob[i] = nums[0]
            elif i is 1:
                rob[i] = max(nums[0],nums[1])
            elif i >= 2:
                rob[i] = max(nums[i]+rob[i-2],rob[i-1])
        return rob[-1]

In [ ]:
# 337. House Robber III
# 二叉树和抢劫问题的集合，同样存在两种情况，一种是不访问根节点时，走的ans1
# 一种是访问根节点时访问的是根节点以及其跨过根节点左右子树再往下的左右子树

# 1) We take the value at that node (meaning we don't take the "take" values from child nodes)
# 2) We skip the value at that node (we can take "take" or "skip" values from child nodes)

# Definition for a binary tree node.
# class TreeNode:
#     def __init__(self, val=0, left=None, right=None):
#         self.val = val
#         self.left = left
#         self.right = right
class Solution:
    def rob(self, root: TreeNode) -> int:
        return self.dfs(root,{})
    def dfs(self,root,memo):
        if not root:
            return 0
        if root in memo:
            return memo[root]
        ans1 = self.dfs(root.left,memo) + self.dfs(root.right,memo)
        
        left,right = 0,0
#         每层进入的都是新的迭代，因此每一层都可以判断自己是skip还是keep
        if root.left:
#             这里如果存在左子树访问的是左子树的左右子树(都需要访问，注意)
            left = self.dfs(root.left.left,memo) + self.dfs(root.left.right,memo)
#     root.left.left也可以skip或者keep，这就进一步进行迭代了
        if root.right:
            right = self.dfs(root.right.right,memo) + self.dfs(root.right.left,memo)
        ans2 = root.val + left + right
        memo[root] = max(ans1,ans2)
        return memo[root]

In [ ]:
# 309. Best Time to Buy and Sell Stock with Cooldown
这道题最重要的是需要推导出状态方程，英文名叫state machine
buy[i]表示在第i天之前最后一个操作是买，此时的最大收益。

sell[i]表示在第i天之前最后一个操作是卖，此时的最大收益。

cooldown[i]表示在第i天之前最后一个操作是冷冻期，此时的最大收益。
注意我们这里标注的都是最大收益

我们可以得到三个式子
1.cooldown[i] = max(sell[i - 1],buy[i-1],cooldown[i-1]) #一共有三种情况下cooldown，每个之后都有可能进行cooldown
2.sell[i] = max(buy[i - 1] + prices[i],sell[i - 1]) #卖操作钱增加，因此是加操作
3.buy[i] = max(rest[i - 1] - prices[i], buy[i-1]) #买操作钱减少，因此是减操作
# 另外，由于冷冻期的存在，我们可以得出cooldown[i] = sell[i-1]，当然这里用原来的式子也是可以的

class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        if not prices:
            return 0 
        
        buy, sell, cooldown = [0 for _ in range(len(prices))], [0 for _ in range(len(prices))], [0 for _ in range(len(prices))]
        
        buy[0] = - prices[0]
        
        for i in range(1, len(prices)):
            cooldown[i] = sell[i - 1]
            sell[i] = max(sell[i - 1], buy[i - 1] + prices[i])
            buy[i] = max(buy[i - 1], cooldown[i - 1] - prices[i])
            
        return max(sell[-1], cooldown[-1])

In [ ]:
# 122. Best Time to Buy and Sell Stock II
# 类似的股票问题，这一题没有上面一题难是因为其没有cooldown过程了，第一反应是和上一题一样写出状态方程，然后实现
# Method 1 复杂度O(n) 
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        if len(prices) == 0:
            return 0
        buy,sell = [0 for i in range(len(prices))],[0 for i in range(len(prices))]
        buy[0] = - prices[0]
        sell[0] = 0
        for i in range(1,len(prices)):
            buy[i] = max(sell[i - 1] - prices[i],buy[i-1])
            sell[i] = max(prices[i] + buy[i - 1],sell[i - 1])
        return sell[-1]
# Method 2 优化空间复杂度，上面要开数组，但是实际上只需要维护一个变量就行了 O(1)空间复杂度
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        if len(prices) <=1:
            return 0
        pre_buy = - prices[0]
        pre_sell = 0
        for i in range(1,len(prices)):
            buy = max(pre_sell - prices[i],pre_buy)
            sell = max(prices[i] + pre_buy,pre_sell)
            pre_buy = buy
            pre_sell = sell
        return sell
# Method 3 
# 我们考虑相邻两天的股票价格，如果后一天的股票价格大于前一天的，那么在进行买入卖出操作后，即可获利。
# 而且这样在不考虑交易次数的前提下，这样的贪心一定能获得最大的利润。
# 贪心算法在有最优子结构的问题中尤为有效。最优子结构的意思是局部最优解能决定全局最优解。
# 简单地说，问题能够分解成子问题来解决，子问题的最优解能递推到最终问题的最优解。
class Solution:
    def maxProfit(self, prices: List[int]) -> int:
        profit = 0
        for i in range(1,len(prices)):
            if prices[i] - prices[i - 1] > 0:
                profit += prices[i] - prices[i - 1]
        return profit
贪心算法与动态规划的不同在于它对每个子问题的解决方案都做出选择，不能回退。
动态规划则会保存以前的运算结果，并根据以前的结果对当前进行选择，有回退功能。

In [ ]:
# 714. Best Time to Buy and Sell Stock with Transaction Fee
现在加入了一个手续费的概念，在有手续费的情况下
(注意不是买卖都存在手续费，而是一整个买卖只存在一笔手续费，因此我们在卖的时候加入手续费的概念，同时用两个list来维护)
class Solution:
    def maxProfit(self, prices: List[int], fee: int) -> int:
        if len(prices) <= 1:
            return 0
        buy,sell = [0 for i in range(len(prices))],[0 for i in range(len(prices))]
        buy[0] = -prices[0]
        for i in range(1,len(prices)):
            buy[i] = max(buy[i - 1], sell[i - 1] - prices[i])
            sell[i] = max(sell[i - 1], buy[i - 1] + prices[i] - fee) #卖的时候考虑手续费
        return sell[-1]
# 这道题同理在买的时候考虑手续费也可以，下面是在买的时候加入手续费的过程
class Solution:
    def maxProfit(self, prices: List[int], fee: int) -> int:
        if len(prices) <= 1:          
            return 0
        buy,sell = [0 for i in range(len(prices))],[0 for i in range(len(prices))]
        buy[0] = - prices[0] - fee
        for i in range(1,len(prices)):
            buy[i] = max(buy[i - 1], sell[i - 1] - prices[i] - fee)
            sell[i] = max(sell[i - 1], buy[i - 1] + prices[i])
        return sell[-1]

In [ ]:
# 343. Integer Break
# Method1 我们的思考顺序是先使用递归的方式，加入记忆化搜索，然后再采用自底向上的动态规划模式进行思思考
# 这道题也是同理，先写出递归的写法再加入记忆化搜索
# 求最值问题，DP适用
# 最优子结构问题
# 求子问题的最优解，可以获得原问题的最优解
class Solution:
    def integerBreak(self, n: int) -> int:
        memo = [-1] * (n+1)
        memo[0] = 1
        return self.DFS(n,memo)
    def DFS(self,n,memo):
        if n == 1:
            return 1
        if memo[n] != -1:
            return memo[n]
        res = -1
        for i in range(1,n):
            # i + n-i
#             尝试不分割i*(n-i)和尝试继续分割
            res = max(res, i*(n-i), i * self.DFS(n - i,memo))
        memo[n] = res # memo[n]
        return res
# Method 2 自底向上的写法,从上面推出来，算法复杂度O(n^2)
class Solution:
    def integerBreak(self, n: int) -> int:
        memo = [-1] * (n+1)
        memo[1] = 1
        for i in range(2, n+1):
#             求解memo[i]
            for j in range(1,i):
#             j +(i - j)
                memo[i] = max(memo[i],j*(i - j),j*memo[i-j])
        return memo[n]

In [ ]:
# 279. Perfect Squares
class Solution:
    def numSquares(self, n: int) -> int:
        dp =[]
        import sys
        for i in range(n+1):
            dp.append(sys.maxsize)
        i = 0
#         先确定平方数字肯定是1
        while i * i <= n:
            dp[i*i] = 1
            i += 1

#          这里min的判断条件是为了排除本身是平方数的情况
        for i in range(n+1):
            j = 1
            while j * j <= i:
                dp[i] = min(dp[i], dp[i-j*j] + 1)
                j += 1
        return dp[n]

In [ ]:
# 91. Decode Ways
class Solution:
    def numDecodings(self, s: str) -> int:
        if not s:
            return 0
        
        return self.dfs(s, 0, {})
        
    def dfs(self, s, start_index, memo):
        if start_index in memo:
            return memo[start_index]
            
        if start_index == len(s): 
            return 1
        
        if s[start_index] == '0':
            return 0 
#状态转移方程
#   - 若s[i - 1]表示的数是1到9，dp[i] += dp[i - 1]
#   - 若s[i - 2]和s[i - 1]表示的数是10到26，dp[i] += dp[i - 2]
#       最后将两种长度的值都加起来
#   - 若上述两种情况都不满足，直接返回答案0
    
        result = self.dfs(s, start_index + 1, memo) #这里是start_index + 1的情况
        
#         这里往下走是start_index + 2的情况，
        if  start_index + 2 <= len(s)  and int(s[start_index : start_index + 2]) <= 26: #要走一遍判断

            result += self.dfs(s, start_index + 2, memo) 
            
        memo[start_index] = result 
        return result

In [ ]:
记忆化搜索的本质 = 动态规划(DP)
三种适用DP的场景
1.求最值(max/min)
2.求方案总数(sum)
3.求可行性(or)
三种不适用DP的场景
1.求所有的具体方案
2.输入数据是无序的(上面这个triangle是只能从上往下的，并不能反向走，如果有反向走这类的可能，就没有办法拿DP做)
3.暴力算法时间复杂度已经是多项式级别，DP就是多项式级别的，不需要进一步的简略了

In [ ]:
# 131. Palindrome Partitioning
# 这道题是不适合DP的场景，这道题要求返回所有的
class Solution:
    def partition(self, s: str) -> List[List[str]]:
        results = []
        self.dfs(s, [], results)
        return results
    def dfs(self,s, string, results):
        if len(s) == 0:
            results.append(list(string))
            return results
        for i in range(1, len(s) + 1):
            now_word = s[:i]
            if self.is_valid(now_word):
                string.append(now_word)
                self.dfs(s[i:], string, results)
                string.pop()
    def is_valid(self,now_word):
        if now_word == now_word[::-1]:
            return True
        return False

In [ ]:
# 128. Longest Consecutive Sequence
# 这道题的输入nums是一个无序的序列，因此是一个不适用与DP的场景，
# 这里我们是用了sort来进行排序就可以使用DP了，但是sort的时间复杂度就已经是O(nlogn)了，因此DP不是最优的解法
class Solution:
    def longestConsecutive(self, nums: List[int]) -> int:
        nums = list(set(nums))
        nums = sorted(nums)
        if len(nums) == 0:
            return 0
        if len(nums) == 1:
            return 1
        print(nums)
        memo = 1
        memo_matrix = []
        for i in range(1,len(nums)):
            if nums[i - 1] + 1 == nums[i]:
                memo += 1
                memo_matrix.append(memo)
            else:
                memo_matrix.append(memo)
                memo = 1
        if len(memo_matrix) == 1:
            return memo_matrix[0]
        return max(memo_matrix)

In [ ]:
# 84. Largest Rectangle in Histogram
# 这种做法对于数据小的时候可以，数据大的时候就会超时
# 现在的时间复杂度是O(n^3),通过栈优化可以优化到O(n^2)
class Solution:
    def largestRectangleArea(self, heights: List[int]) -> int:
        if len(heights) == 0:
            return 0
        # max_one = max(heights[:])
        # print(max_one)
        max_size = -sys.maxsize
        for i in range(len(heights)):
            for j in range(len(heights)- 1, -1,-1):
                if i <= j:
                    # max_size = max(max_one,max(max_size, (min(heights[i:j+1]) * (j - i + 1))))
                    max_size = max(max_size, (min(heights[i:j+1]) * (j - i + 1)))
        return max_size
            
#         枚举左边柱子右边柱子 + 找他们中间最小的值

# 因此要用栈空间来求解

In [ ]:
记忆化搜索的缺陷
递归深度太深，导致StackOverflow
解决方法：用递推方法，数组 (动态规划的第二种写法)
递推的典型标志是其有一个for循环

In [ ]:
适用for循环动态规划的例子有word break，单词从前往后计数，
当我判断第i个位置是否能切分时，我先判断word[i - wordlength]是否是True，如果是True就继续，False证明前面都切分不了
那现在就没有必要继续判断了，直接走下一个。
注意这个wordlength是从最短wordlength(1)，一直到最长wordlength，因此这里也要写一个for循环，总的来说就是双重for
循环嵌套，之前走过的先判断为True了，之后再走的时候如果走到了直接走过判断为True的，才判断剩下部分在不在dict里
否则直接返回False

In [ ]:
# 139. Word Break
# 这道题问的是if，问的是可行性的问题，是可以通过DP进行求解的
# 这道题不能用递归就是自己调自己，因为递归太深就会爆掉
# 这道题采用的是递推+数组的方法
class Solution:
    def wordBreak(self, s: str, wordDict: List[str]) -> bool:
        if not s:
            return True
        if not wordDict:
            return False
        dp = [0] * (len(s) + 1)
        dp[0] = True
        maxlength = max([len(word) for word in wordDict]) if wordDict else 0
        for i in range(1,len(s) + 1):
            for j in range(1,maxlength + 1):
                if dp[i - j] == 0:
                    continue
                if i < 1:
                    break
                word = s[i - j : i]
                if word in wordDict:
                    dp[i] = True
                    break
        return dp[len(s)]    

In [ ]:
# 509. Fibonacci Number
# 通过记忆之前的数，加快自迭代的速度
# 记忆化搜索简单题
# 记忆化搜索-自上向下的解决问题  
class Solution:
    def fib(self, N: int) -> int:
        memo = [-1] * (N + 1)
        if N<= 1:
            return N
        if memo[N] == -1:
            memo[N] = self.fib(N-1) + self.fib(N-2)
        return memo[N]

In [ ]:
# 70. Climbing Stairs
# 斐波那契数列的同种变形题，用到的是递推的思想，如果我们使用递归思想，
# 递归深度过深，因此需要使用memo记忆化搜索
class Solution:
    def climbStairs(self, n: int) -> int:
        memo = [-1] * (n + 1)
        memo[0] = 1
        memo[1] = 1
        for i in range(2,n + 1):
            memo[i] = memo[i - 1] + memo[i - 2]
        return memo[n]

In [ ]:
# 0-1背包问题
# leetcode上并没有最原始的0-1背包问题，但是有其变体
# 416. Partition Equal Subset Sum
# 这道题是非常经典的背包问题变体，这里背包的目标填充是一半的sum空间，如果能填上就满足要求，如果sum是奇数则直接排除
# chrome收藏夹里收藏了一个背包问题的理解
时间复杂度O(n*c)
空间复杂度O(n*c)
class Solution:
    def canPartition(self, nums: List[int]) -> bool:
        if not nums:
            return False 
        W = sum(nums)
        if W % 2 != 0:
            return False 
            
        m = W // 2
        n = len(nums)
        dp = [[False] * (m + 1) for _ in range(n + 1)]
        dp[0][0] = True 
        for i in range(1, n + 1):
            for w in range(m + 1): #w表示的是最大重量限制
#                 最大重量限制从0一直到我们的最大目标值m  =W//2 
                if w >= nums[i - 1]: #最大重量限制比当前输入数值大(证明当前值是有机会输入的，这里分为输入和不输入)
                    dp[i][w] = dp[i - 1][w] or dp[i - 1][w - nums[i - 1]]
                else: #最大重量限制比nums当前值要小，那么nums这个值肯定不能放进去，因此其等于i-1时刻的值
                    dp[i][w] = dp[i - 1][w]
        return dp[n][m]
    
# Method 2
# 注意这里在空间上仍然可以优化，该行的数据只与上一行数据有关，因此理论上只需要保持两行元素
# 空间复杂度变为O(2*c)其中c为行数
# 那么空间复杂度就变为O(c)了
时间复杂度仍然为O(n*c)
上面一行永远处理i为偶数
下面一行永远处理i为奇数
通过对这两行进行轮流的更新，来达到节省空间的目的

class Solution:
    def canPartition(self, nums: List[int]) -> bool:
        if not nums:
            return False 
        W = sum(nums)
        if W % 2 != 0:
            return False 
        m = W // 2
        n = len(nums)
        dp = [[False] * (m + 1) for _ in range(2)]
        dp[0][0] = True 
        for i in range(1, n + 1):
            for w in range(m + 1): #w表示的是最大重量限制
#                 最大重量限制从0一直到我们的最大目标值m  =W//2 
                if w >= nums[(i - 1)%2]: #最大重量限制比当前输入数值大(证明当前值是有机会输入的，这里分为输入和不输入)
                    dp[i%2][w] = dp[(i - 1)%2][w] or dp[(i - 1)%2][w - nums[i - 1]]
                else: #最大重量限制比nums当前值要小，那么nums这个值肯定不能放进去，因此其等于i-1时刻的值
                    dp[i%2][w] = dp[(i - 1)%2][w]
        return dp[n%2][m]
    
# Method 3
# 从右往做更新
9.6 11:42 lxz网课

In [ ]:
# 0-1背包问题还有很多变种问题
多重背包问题：每个物体不止1个，有num(i)个
完全背包问题，每个物体可以无限使用，(可以将其转换成一个有限使用的背包问题)
多为费用背包问题：同时考虑多维度局限性，要考虑物品的体积和重量两个维度
物品间加入更多约束：物品间可以互相排斥，或者互相依赖(存在一个物品另一个物品也需要存在，或者二者只能选取1)

In [ ]:
# 300. Longest Increasing Subsequence
# 最长上升子序列
class Solution:
    def lengthOfLIS(self, nums: List[int]) -> int:
        n = len(nums)
        if n < 1:
            return 0
        res = [1 for i in range(n)]
        for i in range(n):
            for j in range(0,i):
                if nums[j]<nums[i]:
                    res[i] = max(res[i],res[j]+1)
        return max(res)
# 注意在取返回值的时候，并不是取最后的值，res值不是逐一递增的(题目例题是逐一递增的，属于特例)
# 时间复杂度是O(n^2) 比暴力穷举法O((2^n)*n)还是要快得多的，2^n是指对于每个数字都取或者不取，两种可能
# 一共有n个数字，因此是两个相乘来找最长的子序列(暴力求解法)


In [ ]:
# 376. Wiggle Subsequence
# 这道题与上面非常类似
# 注意这道题只要前面满足正负正负变化，数列中是可以跳数的！这就是要维护j的范围是[0:i](i不取的原因)
class Solution:
    def wiggleMaxLength(self, nums: List[int]) -> int:
        if nums==[]:
            return 0

        size = len(nums)
        p = [1]*size
        q = [1]*size
        for i in range(1, size):
            for j in range(i):
                if nums[i]>nums[j]:
                    p[i] = max(p[i], q[j]+1)
                elif nums[i]<nums[j]:
                    q[i] = max(q[i], p[j]+1)
                elif nums[i] == nums[j]:
                    continue

        return max(p[size-1], q[size-1])

In [ ]:
# 1143. Longest Common Subsequence
# 最长公共子序列,通途广泛，如衡量两个的相似度之类的
# 两个字符串进行扫描，因此是两个维度的问题，这里dp要设置成二维的
LCS(M,N)表示S1[0...M]和S2[0...N]的最长公共子序列的长度
S1[M] = S2[N]:
    LCS(M,N) = 1 + LCS(M - 1, N -1)
S1[M] != S2[N]:
    LCS(M,N) = max(LCS(M - 1, N),LCS(M,N - 1))
    #左右两边分别向内缩一下看是否这时存在相同字符，不存在则继续缩
#递归 + 记忆化搜索 (first thought)如果单纯递归没法通过，先放上一个递归版本
# Method 1 Recursion
class Solution:
    def longestCommonSubsequence(self, s1: str, s2: str) -> int:
        return self.helper(s1,s2,0,0)
    def helper(self,s1,s2,i,j):
        if i == len(s1) or j == len(s2):
            return 0
        elif s1[i] == s2[j]:
            return 1 + self.helper(s1,s2,i+1,j+1)
        else:#S1[i]!=S2[j]
            return max(self.helper(s1,s2,i,j+1),self.helper(s1,s2,i+1,j))

# Method 2 Recursion + memoization(Pass)
class Solution:
    def longestCommonSubsequence(self, s1: str, s2: str) -> int:
        memo = [[-1 for _ in range(len(s2) + 1)] for _ in range(len(s1) + 1)]
        return self.helper(s1, s2, len(s1)-1, len(s2) - 1,memo)
            
    def helper(self, s1, s2, i, j,memo):
        if memo[i][j] < 0:            
            if i == -1 or j == -1:
                memo[i][j] = 0
            elif s1[i] == s2[j]:
                memo[i][j] =  1 + self.helper(s1, s2, i - 1, j - 1,memo)
            else:
                memo[i][j] =  max(self.helper(s1, s2, i - 1, j,memo), self.helper(s1, s2, i, j - 1,memo))
        return memo[i][j]
# Method 3 DP 通过memoization可以推出DP写法
class Solution:
    def longestCommonSubsequence(self, s1: str, s2: str) -> int:
        memo = [[0 for _ in range(len(s2) + 1)] for _ in range(len(s1) + 1)]
        for i in range(1,len(s1) + 1):
            for j in range(1,len(s2) + 1):
                if s1[i-1] == s2[j-1]:
                    memo[i][j] =  1 + memo[i - 1][j - 1]
                else:
                    memo[i][j] =  max(memo[i - 1][j],memo[i][j - 1])
        # return memo[len(memo) - 1][len(memo[0]) - 1]
        return memo[-1][-1]

In [ ]:
如何用动态规划求出具体的解？
我们在这之前都是找到了目标长度这样，我们要如何找到具体的子序列呢
我们从后往前推，如果我们知道了目标长度，我们接下来找目标长度 - 1的值 +(满足某条件)
然后再减1一直往前找 + (满足某条件)

In [ ]:
面试中考察算法部分对基础的考察较多，不排除一些大公司出动态规划

In [ ]:
332. Coin Change
class Solution:
    def coinChange(self, coins: List[int], amount: int) -> int:
        dp = [float('inf')] *(amount +1)
        dp[0] = 0
        for i in range(1,amount + 1):
            for coin in coins:
#                 这道题最值得注意的就是这里的判别条件，首先i一定要是大于等于coin值的，否则减出来负值超出范围
#                 另外就是后面这个inf判断用来剪枝，不写也能通过但是更慢一些
                if i >= coin and dp[i - coin] != float('inf'):
                    dp[i] = min(dp[i],dp[i - coin] + 1)
        if dp[amount] != float('inf'):
            return dp[amount]
        return -1

In [ ]:
# 377. Combination Sum IV
# DP写法，也可以使用memoization+DFS
class Solution:xiefq
    def combinationSum4(self, nums: List[int], target: int) -> int:
        dp = [0] * (target+1)
        dp[0] = 1
        nums.sort()

        for i in range(1, target+1):
            for j in range(len(nums)):
                if nums[j] <= i:
                    dp[i] += dp[i - nums[j]]
                else:
                    break

        return dp[target]
# Memoization + DFS 采用计数的方法来写的
class Solution:
    def combinationSum4(self, nums: List[int], target: int) -> int:
        nums.sort()
        memo = {}
        result = self.dfs(nums,target,memo)
        return result
    def dfs(self,nums,target,memo):
        if target == 0:
            return 1
        if target < 0:
            return 0
        if target in memo:
            return memo[target]
        cnt = 0
        for i in range(len(nums)):
            if nums[i] > target:
                continue
            cnt += self.dfs(nums,target - nums[i], memo)
        memo[target] = cnt
        return cnt

In [ ]:
474. Ones and Zeros